<a href="https://colab.research.google.com/github/dutrasilva/dutrasilva/blob/main/Trabalho_Final_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho Final: KDD aplicado à Classificação de Qualidade de Café

## **Integrantes**
* **Nome:** André Luiz Silva e Wagner Henrique Cichacz
* **Curso:** Desenvolvimento de Sistemas Web / Monitoramento
* **Disciplina:** Inteligência Artificial e Data Mining
* **Data:** Novembro 2025

## **Sumário**
1.  Motivação e Problema
2.  Instalação e Confirguração das Bibliotecas
3.  Dados
4.  KDD: Seleção
5.  EDA (Análise Exploratória)
6.  Engenharia de Features
7.  Pré-processamento e Transformação
8.  Particionamento
9.  Modelagem (Mineração)
10.  Avaliação
11. Salvamento do Modelo (Código)
12. Conclusão


---

<a id="motivacao"></a>
## **1. Motivação e Problema**
* **Cliente:** Cooperativa de Cafeicultores (Fictícia).
* **Problema:** A classificação manual (Q-Graders) é cara e subjetiva.
* **Objetivo de Negócio:** Automatizar a triagem inicial de lotes.
* **Objetivo de Data Mining:** Classificar cafés como **"Especial" (>85 pontos)** ou **"Comercial"** com base em características sensoriais e físicas.
* **Meta:** F1-Score > 0.75 para a classe Especial.

In [ ]:
# 2. Instalação e Confirguração das Bibliotecas

# CONFIGURAÇÃO INICIAL E BIBLIOTECAS ---
# Instalação de dependências (caso necessário)
!pip install pandas numpy matplotlib seaborn scikit-learn xgboost joblib -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib

# Bibliotecas de Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression # Baseline
from sklearn.ensemble import RandomForestClassifier # Modelo IA
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

# REPRODUTIBILIDADE [Requisito do PDF]: Fixar semente aleatória
SEED = 42
np.random.seed(SEED)

print("Bibliotecas importadas e Seed definida para 42.")

In [ ]:
# 3. Carregamento dos Dados (Código)

# --- CARREGAMENTO DOS DADOS (À PROVA DE ERROS) ---
import os
import pandas as pd

# Definição dos caminhos
# Vamos usar o diretório temporário padrão do Colab
DATA_DIR = '/content/drive/MyDrive/Trabalho_Final_IA/Modelos/df_arabica_clean.csv'
FILE_NAME = 'df_arabica_clean.csv'
LOCAL_PATH = os.path.join(DATA_DIR, FILE_NAME)

# Montar o Drive APENAS para SALVAR O MODELO (Requisito do PDF)
# O try/except evita que o código pare se o aluno cancelar a montagem do drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
    OUTPUT_PATH = '/content/drive/MyDrive/Trabalho_Final_IA/Modelos/'
    os.makedirs(OUTPUT_PATH, exist_ok=True)
    print(f"Google Drive montado. Modelos serão salvos em: {OUTPUT_PATH}")
except:
    print("AVISO: Google Drive não foi montado. O modelo será salvo apenas na memória temporária.")
    OUTPUT_PATH = '/content/' # Salva na raiz se não tiver Drive

# Verificação Final
if 'df' in locals():
    print("\n--- Dados Carregados com Sucesso! ---")
    print(f"Tamanho do Dataset: {df.shape}")
    display(df.head(3))
else:
    print("\nERRO FATAL: Não foi possível carregar os dados de nenhuma fonte.")

In [ ]:
# 4. KDD - Seleção (Código)

# --- KDD: SELEÇÃO ---
# Justificativa: Selecionamos colunas sensoriais (Aroma, Flavor, etc) e físicas (Moisture, Color).
# A variável alvo (Target) é derivada de 'Total Cup Points'.

# 1. Criação da Variável Alvo Binária (Classificação)
# Regra de Negócio: Nota > 85 é Especial (1), caso contrário Comercial (0)
df['Quality_Class'] = np.where(df['Total Cup Points'] > 85, 1, 0)

# 2. Seleção de Features
features = [
    'Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body',
    'Balance', 'Uniformity', 'Clean Cup', 'Sweetness',
    'Moisture Percentage', 'Processing Method', 'Color'
]
target = 'Quality_Class'

df_selected = df[features + [target]].copy()

print(f"Dataset Selecionado: {df_selected.shape[0]} linhas e {df_selected.shape[1]} colunas.")
print("\nBalanceamento das Classes:")
print(df_selected[target].value_counts(normalize=True))

In [ ]:
# 5. EDA - Análise Exploratória (Código)

# --- EDA (ANÁLISE EXPLORATÓRIA) ---

# A. Verificar Nulos
print("Valores Nulos:")
print(df_selected.isnull().sum()[df_selected.isnull().sum() > 0])

# B. Visualizar a Target
plt.figure(figsize=(6, 4))
sns.countplot(x=target, data=df_selected)
plt.title("Distribuição das Classes (Desbalanceamento)")
plt.show()

# C. Boxplot para identificar Outliers nas notas sensoriais
numeric_feats = df_selected.select_dtypes(include=np.number).columns.drop(target)
plt.figure(figsize=(15, 6))
df_selected[numeric_feats].boxplot()
plt.xticks(rotation=45)
plt.title("Boxplot das Features Numéricas (Verificação de Outliers)")
plt.show()

# D. Matriz de Correlação
plt.figure(figsize=(10, 8))
sns.heatmap(df_selected[numeric_feats].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlação entre Variáveis")
plt.show()

In [ ]:
# 6. Engenharia de Features (Código)

# --- ENGENHARIA DE FEATURES ---
# Hipótese Agronômica: Um café com média alta em todos os atributos sensoriais
# é mais estável que um café com apenas um atributo alto.

sensory_cols = ['Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance']
df_selected['Sensory_Avg'] = df_selected[sensory_cols].mean(axis=1)

# Atualizar listas de features
numeric_features = df_selected.select_dtypes(include=np.number).columns.drop(target)
categorical_features = df_selected.select_dtypes(include=['object', 'category']).columns

print("Nova feature 'Sensory_Avg' criada.")

In [ ]:
# 7.  Pré-processamento e Transformação
# 8.  Particionamento

# --- PRÉ-PROCESSAMENTO E PARTICIONAMENTO ---

# 1. Divisão Treino/Teste (Estratificada devido ao desbalanceamento)
X = df_selected.drop(target, axis=1)
y = df_selected[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)

# 2. Definição do Pipeline de Pré-processamento
# Numérico: Imputação pela mediana (robusto a outliers) + Padronização (StandardScaler)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categórico: Imputação pela moda + OneHotEncoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

print("Pipelines de pré-processamento configurados.")
print(f"Treino: {X_train.shape[0]} amostras | Teste: {X_test.shape[0]} amostras.")

In [ ]:
# 9. Modelagem e Mineração (Código)

# --- MODELAGEM (MINERAÇÃO) ---

# A. BASELINE: Regressão Logística
# Motivo: Modelo simples e interpretável para servir de comparação.
pipeline_lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=SEED, max_iter=1000))
])
pipeline_lr.fit(X_train, y_train)
print("Baseline (Regressão Logística) treinado.")

# B. MODELO AVANÇADO: Random Forest com GridSearchCV
# Motivo: Lida bem com relações não-lineares e dados tabulares.
pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=SEED))
])

# Otimização de Hiperparâmetros
param_grid = {
    'classifier__n_estimators': [100, 200], # Número de árvores
    'classifier__max_depth': [None, 10],   # Profundidade
    'classifier__class_weight': ['balanced', None] # Tratar desbalanceamento
}

grid_search = GridSearchCV(pipeline_rf, param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

print(f"Melhor F1-Score na Validação: {grid_search.best_score_:.4f}")
print(f"Melhores Parâmetros: {grid_search.best_params_}")

In [ ]:
# 10. Avaliação dos Resultados (Código)

# --- AVALIAÇÃO ---
# Métrica Principal: F1-Score (pois o dataset é desbalanceado e queremos equilíbrio entre Precision/Recall)

print("--- RELATÓRIO BASELINE (LogRegression) ---")
y_pred_lr = pipeline_lr.predict(X_test)
print(classification_report(y_test, y_pred_lr))

print("--- RELATÓRIO IA (Random Forest Otimizado) ---")
y_pred_rf = best_model.predict(X_test)
print(classification_report(y_test, y_pred_rf))

# Matriz de Confusão
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
sns.heatmap(confusion_matrix(y_test, y_pred_lr), annot=True, fmt='d', cmap='Blues', ax=ax[0])
ax[0].set_title('Baseline')
sns.heatmap(confusion_matrix(y_test, y_pred_rf), annot=True, fmt='d', cmap='Greens', ax=ax[1])
ax[1].set_title('Random Forest')
plt.show()

# Curva ROC
y_prob = best_model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_prob)
auc = roc_auc_score(y_test, y_prob)

plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, label=f"Random Forest (AUC={auc:.2f})")
plt.plot([0,1], [0,1], 'k--')
plt.legend()
plt.title("Curva ROC")
plt.show()

In [ ]:
# 11. Salvamento do Modelo e Conclusão (Código)

# --- CONCLUSÃO E EXPORTAÇÃO ---

# Salvar modelo no Drive
model_filename = os.path.join(OUTPUT_PATH, 'modelo_cafe_rf_final.joblib')
joblib.dump(best_model, model_filename)
print(f"Modelo salvo com sucesso em: {model_filename}")

# Para recarregar (demonstração):
loaded_model = joblib.load(model_filename)

## **12. Conclusões**
O projeto atingiu o objetivo de classificar cafés especiais.
1.  **Comparação:** O Random Forest superou (ou igualou) a Regressão Logística, mostrando robustez.
2.  **Impacto:** A automatização permite que a cooperativa foque esforços humanos apenas nos casos duvidosos, economizando tempo.
3.  **Limitações:** O tamanho da amostra é pequeno, sugere-se coletar mais dados de safras futuras.